In [2]:
#import the dependencies
import pandas as pd
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
from config import g_key
import gmaps

In [3]:
#store the saved CSV into a DataFrame
city_data_df =pd.read_csv("weather_data/WeatherPy_challenge.csv")

In [4]:
#Configure gmaps to use you Google API key
gmaps.configure(api_key=g_key)

In [5]:
#Get the latitude and longitude
locations = city_data_df[["Lat", "Long"]]

#Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps=[]

In [6]:
#Ask the customer to add a minimum and maximum temperatu
max_temp =re value.float(input("What is the minimum temperature you would like for your trip? ")) 
min_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_flag = input("Do you want it to be raining? (yes/no) ")
snow_flag = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [7]:
#Filter the dataset to find the cities that fit the criteria
if rain_flag=="yes": 
    if snow_flag=="yes":
        preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                             (city_data_df["Max Temp"]<= max_temp) &
                                             (city_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                             (city_data_df["Snow inches (last 3 hrs)"] > 0)]
    else:
        preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                            (city_data_df["Max Temp"]<= max_temp) &
                                            (city_data_df["Rain inches (last 3 hrs)"] > 0) & 
                                            (city_data_df["Snow inches (last 3 hrs)"] == 0)]
        
else:
    if snow_flag == "yes":
        preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                             (city_data_df["Max Temp"]<= max_temp) & 
                                             (city_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                             (city_data_df["Snow inches (last 3 hrs)"] > 0)]  

    else:
        preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                             (city_data_df["Max Temp"]<= max_temp) & 
                                             (city_data_df["Rain inches (last 3 hrs)"] == 0) & 
                                             (city_data_df["Snow inches (last 3 hrs)"] == 0)]

In [8]:
#Create DataFram called hotel_df to store hotel names along with city, country,max temp, and coordinates
hotels_df = preferred_city_df[["City", "Country", "Max Temp", "Lat", "Long", "Current Description"]].copy()
hotels_df["Hotel Name"] =""

In [9]:
#Set parameters to search for a hotel
params ={
        "radius":5000,
        "type":"lodging",
        "key":g_key,
        "location":","}

In [10]:
#Iterate through the DataFrame
for index, row in hotels_df.iterrows():
    #Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Long"]
    
    #Add the latitude and longitude to location key for the params dictionary
    params["location"]=f"{lat},{lng}"
    
    #Use the search terms: :"lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params).json()
    
    #Grab the first hotel from the result and store the name
    try:
        hotels_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [11]:
#Add a heatmap of temperature for the vacation spots
locations = hotels_df[["Lat", "Long"]]
max_temp = hotels_df["Max Temp"]
info_box_template= "<dl><dt>Hotel Name</dt><dd>{Hotel Name}</dd><dt>City</dt><dd>{City}</dd><dt>Country</dt><dd>{Country}</dd><dt>Weather</dt><dd>{Current Description}</dd><dt>Max Temp</dt><dd>{Max Temp}F</dd></dl>"
hotels_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]

#Assign the figure variable
fig =gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
                                 dissipating =False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotels_info)

#Add the heatmap layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#Create the output file (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"

#Export the City data into a CSV
hotels_df.to_csv(output_data_file, index_label ="City_ID")